In [1]:
import keras
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas
from keras import Sequential
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import SGD, Adam
from keras.callbacks import LearningRateScheduler
import keras.callbacks as callbacks

Using TensorFlow backend.


input1 = Missing image (Masked by cloud)

input2 = Referenced images (cloud-free)

### Implement Model

In [2]:
input1 = Input(shape=(400,400,1), name='input1')
input2 = Input(shape=(400,400,1), name='input2')
input3 = Input(shape=(400,400,1), name='input3')

conv_1 = Conv2D(30, (3, 3), padding='same')(input1)
conv_2 = Conv2D(30, (3, 3), padding='same')(input2)
conv_3 = Conv2D(60, (3, 3), padding='same')(input3)

concat_1_2 = concatenate([conv_1, conv_2], axis=-1)
concat_1_2 = Activation('relu')(concat_1_2)

feature_3 = Conv2D(filters=20, kernel_size=(3, 3),padding='same')(concat_1_2)
feature_5 = Conv2D(filters=20, kernel_size=(5, 5),padding='same')(concat_1_2)
feature_7 = Conv2D(filters=20, kernel_size=(7, 7),padding='same')(concat_1_2)
feature_3_5_7 = concatenate([feature_3, feature_5, feature_7])
feature_3_5_7 = Activation('relu')(feature_3_5_7)

feature_3_5_7 = Dropout(0.5)(feature_3_5_7)

sum0 = add([concat_1_2, feature_3_5_7])

conv1 = Conv2D(filters=60, kernel_size=(3,3), padding='same',activation='relu')(sum0)
conv2 = Conv2D(filters=60, kernel_size=(3,3), padding='same',activation='relu')(conv1)

sum1 = add([conv2, conv_3])
conv3 = Conv2D(filters=60, kernel_size=(3,3), dilation_rate=2, padding='same',activation='relu')(sum1)
conv4 = Conv2D(filters=60, kernel_size=(3,3), dilation_rate=3, padding='same',activation='relu')(conv3)
conv5 = Conv2D(filters=60, kernel_size=(3,3), dilation_rate=2, padding='same',activation='relu')(conv4)
sum2 = add([conv5, conv_3])

conv6 = Conv2D(filters=60, kernel_size=(3,3), padding='same',activation='relu')(sum2)

conv6 = Dropout(0.5)(conv6)

conv6_2 = add([conv6, conv1])

conv7 = Conv2D(filters=1, kernel_size=(3,3), padding='same')(conv6_2)

model = Model([input1, input2, input3], conv7)
    

#### Define PSNR Loss Metrics, remains loss function MSE ; Step Decay for Callbacks

In [3]:
def PSNRLoss(y_true, y_pred):
    return -10. * K.log(K.mean(K.square(y_pred - y_true))) / K.log(10.)

In [4]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.3
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

## Read dataset

In [5]:
import h5py

f = h5py.File('small_train.hdf5', 'r')
trainY = f['label'][:]
trainX_1 = f['masked'][:]
trainX_2 = f['ref'][:]
trainX_3 = f['mask'][:]
f.close()


f = h5py.File('small_test.hdf5', 'r')
testY = f['label'][:]
testX_1 = f['masked'][:]
testX_2 = f['ref'][:]
testX_3 = f['mask'][:]
f.close()

In [6]:
trainX_3[:,:,:] = (765 - trainX_3[:,:,:])
trainX_3[:,:,:] = trainX_3[:,:,:] // 765
trainX_3 = abs(trainX_3)

In [7]:
testX_3[:,:,:] = (765 - testX_3[:,:,:])
testX_3[:,:,:] = testX_3[:,:,:] // 765
testX_3 = abs(testX_3)

In [8]:
img_rows, img_cols = 400, 400
out_rows, out_cols = 400, 400

trainX_3 = trainX_1 + np.multiply(trainX_2, np.reshape(trainX_3, (trainX_3.shape[0], img_rows, img_cols, 1)))
testX_3 = testX_1 + np.multiply(testX_2, np.reshape(testX_3, (testX_3.shape[0], img_rows, img_cols, 1)))

### Config & train model

In [9]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
model.compile(optimizer=adam, loss='mean_squared_error', metrics=[PSNRLoss])

lrate = LearningRateScheduler(step_decay)
callback_list = [callbacks.ModelCheckpoint('weight_20.h5', monitor='val_PSNRLoss', save_best_only=True,
                                           mode='max', save_weights_only=True, verbose=1)]

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
batch_size = 16
nb_epoch = 200

history = model.fit([trainX_1, trainX_2, trainX_3], trainY, batch_size=batch_size, epochs=nb_epoch, callbacks=callback_list,
                    verbose=1, validation_split=0.1)  

Train on 729 samples, validate on 81 samples
Epoch 1/200
729/729 [==============================] - 39s 54ms/step - loss: 0.0023 - PSNRLoss: 27.2114 - val_loss: 3.0018e-06 - val_PSNRLoss: 55.2261
Epoch 2/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0019 - PSNRLoss: 28.4165 - val_loss: 2.6628e-06 - val_PSNRLoss: 55.7467
Epoch 3/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0017 - PSNRLoss: 28.4614 - val_loss: 3.9625e-06 - val_PSNRLoss: 54.0203
Epoch 4/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0016 - PSNRLoss: 28.6563 - val_loss: 8.3593e-06 - val_PSNRLoss: 50.7783
Epoch 5/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0016 - PSNRLoss: 28.6761 - val_loss: 6.7926e-06 - val_PSNRLoss: 51.6797
Epoch 6/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0014 - PSNRLoss: 29.0738 - val_loss: 2.6599e-05 - val_PSNRLoss: 45.7513
Epoch 7/200
729/729 [============================

Epoch 30/200
729/729 [==============================] - 36s 49ms/step - loss: 9.7189e-04 - PSNRLoss: 30.7258 - val_loss: 1.2192e-06 - val_PSNRLoss: 59.1391
Epoch 31/200
729/729 [==============================] - 36s 49ms/step - loss: 9.7481e-04 - PSNRLoss: 30.9204 - val_loss: 2.7851e-06 - val_PSNRLoss: 55.5516
Epoch 32/200
729/729 [==============================] - 36s 49ms/step - loss: 9.5227e-04 - PSNRLoss: 32.0587 - val_loss: 1.1082e-06 - val_PSNRLoss: 59.5538
Epoch 33/200
729/729 [==============================] - 36s 49ms/step - loss: 9.6008e-04 - PSNRLoss: 31.5379 - val_loss: 6.2030e-07 - val_PSNRLoss: 62.0739
Epoch 34/200
729/729 [==============================] - 36s 49ms/step - loss: 9.5612e-04 - PSNRLoss: 30.7507 - val_loss: 7.4854e-07 - val_PSNRLoss: 61.2578
Epoch 35/200
729/729 [==============================] - 36s 49ms/step - loss: 9.6137e-04 - PSNRLoss: 30.6815 - val_loss: 2.9179e-07 - val_PSNRLoss: 65.3492
Epoch 36/200
729/729 [==============================] - 36s 49ms

Epoch 59/200
729/729 [==============================] - 36s 49ms/step - loss: 8.1233e-04 - PSNRLoss: 31.4441 - val_loss: 3.7208e-07 - val_PSNRLoss: 64.2936
Epoch 60/200
729/729 [==============================] - 36s 49ms/step - loss: 7.7543e-04 - PSNRLoss: 32.3480 - val_loss: 1.5045e-06 - val_PSNRLoss: 58.2259
Epoch 61/200
729/729 [==============================] - 36s 49ms/step - loss: 7.8560e-04 - PSNRLoss: 31.6646 - val_loss: 6.7564e-06 - val_PSNRLoss: 51.7029
Epoch 62/200
729/729 [==============================] - 36s 49ms/step - loss: 7.7031e-04 - PSNRLoss: 33.7515 - val_loss: 6.5618e-07 - val_PSNRLoss: 61.8297
Epoch 63/200
729/729 [==============================] - 36s 49ms/step - loss: 7.8338e-04 - PSNRLoss: 32.7645 - val_loss: 4.8663e-06 - val_PSNRLoss: 53.1280
Epoch 64/200
729/729 [==============================] - 36s 49ms/step - loss: 7.3592e-04 - PSNRLoss: 32.3825 - val_loss: 3.0464e-06 - val_PSNRLoss: 55.1621
Epoch 65/200
729/729 [==============================] - 36s 49ms

Epoch 88/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0013 - PSNRLoss: 29.9306 - val_loss: 1.6252e-05 - val_PSNRLoss: 47.8908
Epoch 89/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0013 - PSNRLoss: 30.6999 - val_loss: 1.3708e-06 - val_PSNRLoss: 58.6303
Epoch 90/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0013 - PSNRLoss: 29.9442 - val_loss: 7.5630e-07 - val_PSNRLoss: 61.2130
Epoch 91/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0012 - PSNRLoss: 30.1936 - val_loss: 1.7691e-06 - val_PSNRLoss: 57.5225
Epoch 92/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0012 - PSNRLoss: 29.8977 - val_loss: 2.6328e-05 - val_PSNRLoss: 45.7959
Epoch 93/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0012 - PSNRLoss: 30.1777 - val_loss: 1.2059e-04 - val_PSNRLoss: 39.1868
Epoch 94/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0012 - P

729/729 [==============================] - 36s 49ms/step - loss: 0.0011 - PSNRLoss: 30.9493 - val_loss: 6.5494e-07 - val_PSNRLoss: 61.8380
Epoch 118/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0011 - PSNRLoss: 30.8333 - val_loss: 6.1588e-07 - val_PSNRLoss: 62.1051
Epoch 119/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0011 - PSNRLoss: 30.2631 - val_loss: 2.5658e-06 - val_PSNRLoss: 55.9078
Epoch 120/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0011 - PSNRLoss: 30.7778 - val_loss: 2.9273e-05 - val_PSNRLoss: 45.3352
Epoch 121/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0011 - PSNRLoss: 30.1787 - val_loss: 1.2296e-07 - val_PSNRLoss: 69.1022
Epoch 122/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0011 - PSNRLoss: 30.1173 - val_loss: 1.7387e-07 - val_PSNRLoss: 67.5977
Epoch 123/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0011 - PSNRLoss

729/729 [==============================] - 36s 49ms/step - loss: 0.0010 - PSNRLoss: 30.6878 - val_loss: 3.1257e-07 - val_PSNRLoss: 65.0505
Epoch 147/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0010 - PSNRLoss: 30.5064 - val_loss: 1.3343e-06 - val_PSNRLoss: 58.7474
Epoch 148/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0010 - PSNRLoss: 30.8673 - val_loss: 3.4410e-07 - val_PSNRLoss: 64.6332
Epoch 149/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0010 - PSNRLoss: 30.4534 - val_loss: 3.6042e-06 - val_PSNRLoss: 54.4319
Epoch 150/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0010 - PSNRLoss: 30.7701 - val_loss: 2.5123e-06 - val_PSNRLoss: 55.9993
Epoch 151/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0010 - PSNRLoss: 30.3500 - val_loss: 1.5428e-06 - val_PSNRLoss: 58.1169
Epoch 152/200
729/729 [==============================] - 36s 49ms/step - loss: 0.0010 - PSNRLoss

Epoch 175/200
729/729 [==============================] - 36s 49ms/step - loss: 9.7193e-04 - PSNRLoss: 31.2742 - val_loss: 8.0355e-08 - val_PSNRLoss: 70.9499
Epoch 176/200
729/729 [==============================] - 36s 49ms/step - loss: 9.9209e-04 - PSNRLoss: 32.6182 - val_loss: 9.2583e-07 - val_PSNRLoss: 60.3347
Epoch 177/200
729/729 [==============================] - 36s 49ms/step - loss: 9.8276e-04 - PSNRLoss: 31.0765 - val_loss: 5.9128e-07 - val_PSNRLoss: 62.2821
Epoch 178/200
729/729 [==============================] - 36s 49ms/step - loss: 9.6653e-04 - PSNRLoss: 31.2613 - val_loss: 4.9837e-08 - val_PSNRLoss: 73.0245
Epoch 179/200
729/729 [==============================] - 36s 49ms/step - loss: 9.6898e-04 - PSNRLoss: 31.4229 - val_loss: 1.6341e-06 - val_PSNRLoss: 57.8671
Epoch 180/200
729/729 [==============================] - 36s 49ms/step - loss: 9.7582e-04 - PSNRLoss: 31.3333 - val_loss: 2.6609e-06 - val_PSNRLoss: 55.7497
Epoch 181/200
729/729 [==============================] - 3

In [ ]:
validation_value = model.evaluate(x=[testX_1, testX_2, testX_3], y=testY, batch_size=8)

In [ ]:
validation_value

In [ ]:
y_pred = model.predict([testX_1[10:11], testX_2[10:11], testX_3[10:11]])

Original Input

In [ ]:
plt.imshow(testY[0,:,:,0])

Predicted

In [ ]:
plt.imshow(y_pred[0,:,:,0])

In [ ]:
print(history.history.keys())

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('MSE')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['PSNRLoss'])
plt.plot(history.history['val_PSNRLoss'])
plt.title('PNSR loss')
plt.ylabel('PSNR')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save_weights('final_small_weight_b16.h5')